# Kaggle
## Competição DSA de Machine Learning - Dezembro 2019

Versão 1.0.0: LB = 0.48866 CV = 0.463102
- modelo: LightGBM (com algumas otimizações)
- features engineering: gerado através do Auto_ViML

Versão 1.0.1: LB = 0.48991 CV = 0.462946
- modelo: LightGBM (com algumas otimizações)
- features engineering: gerado através do Auto_ViML (com novas features)

Versão 1.0.2: LB = 0.48915 CV = 0.464442
- modelo: LightGBM (com algumas otimizações)
- features engineering: gerado através do Auto_ViML (com agrupamento pela coluna v2)

Versão 1.0.3: LB = 0.88299 CV = 0.344659
- modelo: LightGBM (com algumas otimizações)
- features engineering: gerado interno

Versão 1.0.4: LB = 0.6041 CV = 0.461724
- modelo: LightGBM (com algumas otimizações)
- features engineering: gerado através do Auto_ViML (modificado v1)

Versão 1.0.5: LB = 0.48897 CV = 0.466055 (kfold)   CV = 0.466422 (StratifiedKFold)
- modelo: LightGBM (com algumas otimizações)
- features engineering: gerado com pacotes estatisticos

Versão 1.0.6: LB = ???? CV = 0.466138 (kfold)   CV =  (StratifiedKFold)
- modelo: LightGBM (com algumas otimizações)
- features engineering: Kernel_Feature_Engineering_v.1.0.0

# Importando as bibliotecas

In [1]:
# Importar os principais pacotes
import numpy as np
import pandas as pd
import itertools
import seaborn as sns
sns.set()

import matplotlib.pyplot as plt
%matplotlib inline

import time
import datetime
import gc

# Evitar que aparece os warnings
import warnings
warnings.filterwarnings("ignore")

# Seta algumas opções no Jupyter para exibição dos datasets
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

# Variavel para controlar o treinamento no Kaggle
TRAIN_OFFLINE = True

In [2]:
# Importa os pacotes de algoritmos
import lightgbm as lgb
from lightgbm.sklearn import LGBMClassifier

# Importa pacotes do sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, log_loss
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder


In [3]:
from sklearn.base import BaseEstimator,TransformerMixin, ClassifierMixin
from sklearn.linear_model import ElasticNetCV, LassoLarsCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline, make_union
from sklearn.utils import check_array
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor

# Carregando os dados de treino e teste

In [4]:
def read_data():
    
    print('Carregando arquivo dataset_treino_new.csv....')
    train = pd.read_csv('../dataset/dataset_treino_new.csv')
    print('dataset_treino.csv tem {} linhas and {} colunas'.format(train.shape[0], train.shape[1]))

    print('Carregando arquivo dataset_teste_new.csv....')
    test = pd.read_csv('../dataset/dataset_teste_new.csv')
    print('dataset_teste.csv tem {} linhas and {} colunas'.format(test.shape[0], test.shape[1]))
    
    return train, test

In [5]:
# Leitura dos dados
train, test = read_data()

Carregando arquivo dataset_treino_new.csv....
dataset_treino.csv tem 114321 linhas and 184 colunas
Carregando arquivo dataset_teste_new.csv....
dataset_teste.csv tem 114393 linhas and 183 colunas


In [6]:
train.head()

,ID,v1,v2,v3,v4,v5,v6,v7,v9,v10,v11,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v26,v27,v28,v29,v30,v31,v35,v36,v37,v39,v40,v42,v44,v45,v47,v48_x,v50,v51,v52,v56,v57,v58,v59,v61,v62_x,v66,v68,v69,v70,v71,v72,v74,v75,v78,v79,v80,v81,v82,v84,v85_x,v86,v88,v90,v91,v92,v93,v94,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119_x,v120,v121,v122,v123_x,v124,v125,v126,v127,v129,v130,v131,1,v62_y,v119_y,v123_y,v48_y,v85_y,v62^2,v62 v119,v62 v123,v62 v48,v62 v85,v119^2,v119 v123,v119 v48,v119 v85,v123^2,v123 v48,v123 v85,v48^2,v48 v85,v85^2,pca_1,ica_1,tsvd_1,grp_1,srp_1,pca_2,ica_2,tsvd_2,grp_2,srp_2,pca_3,ica_3,tsvd_3,grp_3,srp_3,pca_4,ica_4,tsvd_4,grp_4,srp_4,pca_5,ica_5,tsvd_5,grp_5,srp_5,pca_6,ica_6,tsvd_6,grp_6,srp_6,pca_7,ica_7,tsvd_7,grp_7,srp_7,pca_8,ica_8,tsvd_8,grp_8,srp_8,pca_9,ica_9,tsvd_9,grp_9,srp_9,pca_10,ica_10,tsvd_10,grp_10,srp_10,pca_11,ica_11,tsvd_11,grp_11,srp_11,pca_12,ica_12,tsvd_12,grp_12,srp_12,target
0,3,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,9.999999,0.503281,16.434109,11.636387,1.355013,8.571428,3.670350,0.106720,0.148883,18.869284,7.730923,XDX,-1.716131e-08,C,1.720818,3.393503,0.590122,8.880867,C,A,8.375452,11.326591,0.454546,4.012088,7.711453,12.707581,10.498338,9.848672,C,12.171734,0.899420,7.277793,G,DI,3.971118,0.529802,10.890984,15.858151,1,C,15.231789,17.142857,11.784549,F,1,B,D,8.571429,E,3.000000,7.528326,8.861647,1.299638,1.707317,0.866426,3.321300,0.905342,A,0.442252,5.814018,3.517721,8.877414,1.191337,19.470200,8.389236,2.757375,4.374296,1.574039,0.007294,12.579185,E,2.382692,3.930922,B,0.433213,O,NaN,15.634908,2.857144,1.951220,6.592012,5.909091,-6.297423e-07,1.059603,0.803572,8.000000,1.989780,0.035754,AU,1.804126,3.113719,0,0.636364,2.857144,1.0,1.0,-6.297423e-07,1.989780,12.171734,1.707317,1.0,-6.297423e-07,1.989780,12.171734,1.707317,3.965754e-13,-1.253049e-06,-0.000008,-1.075170e-06,3.959225,24.219074,3.397185,148.15110,20.781006,2.914931,-48.164520,-0.003843,153.40079,-91.710680,-1.847689,1.603576,-0.000946,-62.110138,49.214336,-31.847212,-0.998099,0.002434,-3.816818,39.819320,9.526614,-14.700655,-0.001772,7.057063,6.008493,-133.76234,-1.278328,-0.000151,-13.019275,0.239730,5.287480,8.710470,0.003242,-1.451880,-2.993159,17.142406,2.471012,-0.000800,8.745543,-28.929620,-20.641521,3.199084,-0.000453,2.470521,74.760445,-0.262313,-0.098913,-0.001254,3.073173,96.09949,-36.283512,-0.424896,-0.001623,0.464463,27.716370,-2.408795,2.754712,0.000137,1.347929,54.711456,-29.066944,-1.715571,0.000271,2.678761,120.17605,22.876034,1
1,4,1.469550,7.023803,C,4.205991,9.191265,2.412790,2.452166,9.059583,1.312910,15.495952,11.636386,1.992031,4.932127,3.554267,0.773906,0.198655,18.036585,6.763110,GUV,1.845672e-07,C,1.826276,2.673322,5.043831,8.296139,C,A,8.068506,14.579479,0.642856,0.378418,14.305766,12.934363,10.782008,9.156046,E,12.411608,1.379210,7.134018,G,DY,4.067039,5.330551,10.535108,15.075894,2,A,18.274548,9.516129,12.053353,F,2,B,D,13.333334,D,2.090909,7.277655,3.430692,1.423294,2.594937,1.158301,1.761547,0.969183,B,0.542669,5.301047,3.743106,8.303966,1.235546,14.475939,6.623713,2.462898,5.125846,2.512034,1.505335,12.085176,B,1.825361,4.247858,A,3.108809,U,G,10.308044,10.476191,2.222223,10.595357,8.136964,1.461561e+00,1.144708,2.436195,6.749117,2.739240,0.598896,AF,1.614802,2.963620,0,1.560137,1.589403,1.0,2.0,1.461561e+00,2.739240,12.411608,2.594937,4.0,2.923122e+00,5.478479,24.823215,5.189874,2.136160e+00,4.003565e+00,18.140322,3.792658e+00,7.503433,33.998367,7.108153,154.04800,32.207340,6.733697,-25.950914,-0.000614,171.01410,-93.247940,0.007697,5.677019,0.000371,-48.366608,48.573597,-27.310768,-0.396850,-0.003999,-0.820012,45.229984,18.562830,-0.618112,-0.000468,1.806945,7.246580,-157.43951,12.524258,-0.000084,0.119447,-17.264480,14.462655,1.614324,-0.000147,12.426791,0.030932,9.338835,-1.088056,0.000706,1.585607,-14.308875,-19.964195,-5.308384,-0.000207,-1.087751,94.821140,-7.352732,1.685836,0.001949,-5.426009,91.57980,-3

In [7]:
test.head()

,ID,v1,v2,v3,v4,v5,v6,v7,v9,v10,v11,v14,v15,v16,v17,v18,v19,v20,v21,v22,v23,v24,v26,v27,v28,v29,v30,v31,v35,v36,v37,v39,v40,v42,v44,v45,v47,v48_x,v50,v51,v52,v56,v57,v58,v59,v61,v62_x,v66,v68,v69,v70,v71,v72,v74,v75,v78,v79,v80,v81,v82,v84,v85_x,v86,v88,v90,v91,v92,v93,v94,v98,v99,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119_x,v120,v121,v122,v123_x,v124,v125,v126,v127,v129,v130,v131,1,v62_y,v119_y,v123_y,v48_y,v85_y,v62^2,v62 v119,v62 v123,v62 v48,v62 v85,v119^2,v119 v123,v119 v48,v119 v85,v123^2,v123 v48,v123 v85,v48^2,v48 v85,v85^2,pca_1,ica_1,tsvd_1,grp_1,srp_1,pca_2,ica_2,tsvd_2,grp_2,srp_2,pca_3,ica_3,tsvd_3,grp_3,srp_3,pca_4,ica_4,tsvd_4,grp_4,srp_4,pca_5,ica_5,tsvd_5,grp_5,srp_5,pca_6,ica_6,tsvd_6,grp_6,srp_6,pca_7,ica_7,tsvd_7,grp_7,srp_7,pca_8,ica_8,tsvd_8,grp_8,srp_8,pca_9,ica_9,tsvd_9,grp_9,srp_9,pca_10,ica_10,tsvd_10,grp_10,srp_10,pca_11,ica_11,tsvd_11,grp_11,srp_11,pca_12,ica_12,tsvd_12,grp_12,srp_12
0,0,1.375466e+00,11.361141,C,4.200779,6.577000,2.081784,1.784386,9.523810,1.312911,16.851852,12.230137,1.773050,2.857142,3.493816,0.096826,0.225481,17.910227,7.437806,AFPB,9.558311e-07,E,0.954152,4.386617,0.573017,6.988848,NaN,A,8.178438,12.183225,8.333339e-01,2.842768e+00,11.754576,10.855019,10.613378,9.417172,J,12.774528,0.118847,8.590754,C,AF,2.602230,2.919712e-01,9.038609,14.911512,1,A,15.912410,8.571429,11.222210,F,1,B,D,15.714287,Q,1.904761,8.918731,8.750730,0.892192,1.860465,-0.892193,1.784387,0.988980,B,0.658269,4.929298,2.992365,10.013503,0.817844,19.708030,4.186787,1.873945,4.129022,1.701894,0.004535,11.688117,B,1.502480,2.121710,A,3.568773,U,NaN,12.246697,11.428572,1.276596,4.777902,5.833334,-9.974950e-07,1.459854,3.267327,7.619049,1.815242,1.112271e-07,AF,1.292368,3.903345,0,2.333334,1.428572e+00,1.0,1.0,-9.974950e-07,1.815242,12.774528,1.860465,1.0,-9.974950e-07,1.815242,12.774528,1.860465,9.949963e-13,-1.810694e-06,-1.274253e-05,-1.855805e-06,3.295102,23.188852,3.377193,163.18855,23.766562,3.461330,-43.090996,0.003600,165.949050,-94.283050,-1.181863,14.907422,2.152697e-03,-67.076195,56.251858,-33.438740,-6.539289,0.001528,-3.657812,46.642147,14.745598,16.238370,0.002693,-0.006453,13.278002,-148.22202,-2.232993,-0.000602,-15.922690,-8.632625,8.774817,11.716435,0.000374,-2.333419,-1.473833,5.315560,2.886478,-0.000055,12.326039,-27.139492,-27.296959,-1.688582,-0.000760,2.898868,76.32178,-12.292347,1.617323,0.000302,-2.432899,99.61218,-35.725174,-4.028471,-0.000584,-0.224993,29.321370,1.358879,0.843256,0.003643,3.718195,50.964745,-25.141129,-2.090111,0.004080,-0.255219,128.247440,27.437662
1,1,1.462450e+00,6.985397,C,4.202803,8.648951,2.410546,2.450766,9.074074,1.291029,15.491329,10.760797,1.989654,4.927536,3.553225,0.780974,0.198826,18.038157,5.895192,FOG,1.881887e-07,D,1.823501,2.672674,5.050246,8.301887,A,A,8.066451,13.741398,6.411843e-01,3.771082e-01,6.737574,12.941175,10.772841,9.146280,C,12.413530,1.027070,7.134889,A,DI,4.067490,5.342586e+00,10.511385,15.073380,1,C,18.262758,9.512196,12.494872,F,5,B,D,13.333334,E,2.091503,7.528326,3.694162,1.423730,2.581794,1.156678,1.765145,0.968483,C,0.543240,5.295964,3.743987,7.637302,1.234859,14.447717,6.617364,2.464835,5.127422,2.511355,0.240808,12.079808,D,1.973861,3.064432,B,3.105476,E,NaN,16.284937,10.482759,2.223665,8.025670,8.141593,1.458136e+00,1.146726,2.441534,6.752577,2.736875,1.397066e-01,I,1.613876,2.970825,0,1.561753,1.584158e+00,1.0,1.0,1.461561e+00,2.739240,12.411608,2.594937,1.0,1.461561e+00,2.739240,12.411608,2.594937,2.136160e+00,4.003565e+00,1.814032e+01,3.792658e+00,7.503433,33.998367,7.108153,154.04800,32.207340,6.733697,-26.447410,-0.000511,169.872000,-92.774710,0.867650,5.102041,-5.940204e-07,-47.599854,46.081764,-29.680504,-0.113162,-0.003313,-0.169563,42.504190,16.592268,0.472029,0.000720,0.589930,9.494712,-157.44410,0.048646,-0.000549,-0.318261,-10.827558,14.469283,0.887762,0.000387,0.025990,1.701538,9.322438,0.538701,-0.000364,0.883599,-20.866108,-25.016594,4.150036,0.000710,0.539900,

In [8]:
df = train.append(test)

In [9]:
df = df.drop(columns = ['1'], axis = 1)

In [10]:
# Verificar a quantidade de features numericas e categoricas

numerical_feats = df.dtypes[df.dtypes != "object"].index
print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = df.dtypes[df.dtypes == "object"].index
print("Number of Categorical features: ", len(categorical_feats))

Number of Numerical features:  164
Number of Categorical features:  19


In [11]:
for c in df.columns:
    col_type = df[c].dtype
    if col_type == 'object' or col_type.name == 'category':
        df[c] =df[c].astype('category')

In [12]:
df.head()

,ID,grp_1,grp_10,grp_11,grp_12,grp_2,grp_3,grp_4,grp_5,grp_6,grp_7,grp_8,grp_9,ica_1,ica_10,ica_11,ica_12,ica_2,ica_3,ica_4,ica_5,ica_6,ica_7,ica_8,ica_9,pca_1,pca_10,pca_11,pca_12,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,srp_1,srp_10,srp_11,srp_12,srp_2,srp_3,srp_4,srp_5,srp_6,srp_7,srp_8,srp_9,target,tsvd_1,tsvd_10,tsvd_11,tsvd_12,tsvd_2,tsvd_3,tsvd_4,tsvd_5,tsvd_6,tsvd_7,tsvd_8,tsvd_9,v1,v10,v100,v101,v102,v103,v104,v105,v106,v107,v108,v109,v11,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119 v123,v119 v48,v119 v85,v119^2,v119_x,v119_y,v120,v121,v122,v123 v48,v123 v85,v123^2,v123_x,v123_y,v124,v125,v126,v127,v129,v130,v131,v14,v15,v16,v17,v18,v19,v2,v20,v21,v22,v23,v24,v26,v27,v28,v29,v3,v30,v31,v35,v36,v37,v39,v4,v40,v42,v44,v45,v47,v48 v85,v48^2,v48_x,v48_y,v5,v50,v51,v52,v56,v57,v58,v59,v6,v61,v62 v119,v62 v123,v62 v48,v62 v85,v62^2,v62_x,v62_y,v66,v68,v69,v7,v70,v71,v72,v74,v75,v78,v79,v80,v81,v82,v84,v85^2,v85_x,v85_y,v86,v88,v9,v90,v91,v92,v93,v94,v98,v99
0,3,-91.710680,27.716370,54.711456,120.17605,49.214336,39.819320,6.008493,0.239730,-2.993159,-28.929620,74.760445,96.09949,-0.003843,-0.001623,0.000137,0.000271,-0.000946,0.002434,-0.001772,-0.000151,0.003242,-0.000800,-0.000453,-0.001254,-48.164520,-0.424896,2.754712,-1.715571,1.603576,-0.998099,-14.700655,-1.278328,8.710470,2.471012,3.199084,-0.098913,-1.847689,-2.408795,-29.066944,22.876034,-31.847212,9.526614,-133.76234,5.287480,17.142406,-20.641521,-0.262313,-36.283512,1.0,153.40079,0.464463,1.347929,2.678761,-62.110138,-3.816818,7.057063,-13.019275,-1.451880,8.745543,2.470521,3.073173,1.335739,0.503281,19.470200,8.389236,2.757375,4.374296,1.574039,0.007294,12.579185,E,2.382692,3.930922,16.434109,B,0.433213,O,NaN,15.634908,2.857144,1.951220,6.592012,5.909091,-1.253049e-06,-0.000008,-1.075170e-06,3.965754e-13,-6.297423e-07,-6.297423e-07,1.059603,0.803572,8.000000,24.219074,3.397185,3.959225,1.989780,1.989780,0.035754,AU,1.804126,3.113719,0,0.636364,2.857144,11.636387,1.355013,8.571428,3.670350,0.106720,0.148883,8.727474,18.869284,7.730923,XDX,-1.716131e-08,C,1.720818,3.393503,0.590122,8.880867,C,C,A,8.375452,11.326591,0.454546,4.012088,3.921026,7.711453,12.707581,10.498338,9.848672,C,20.781006,148.15110,12.171734,12.171734,7.915266,0.899420,7.277793,G,DI,3.971118,0.529802,10.890984,2.599278,15.858151,-6.297423e-07,1.989780,12.171734,1.707317,1.0,1,1.0,C,15.231789,17.142857,3.176895,11.784549,F,1,B,D,8.571429,E,3.000000,7.528326,8.861647,1.299638,2.914931,1.707317,1.707317,0.866426,3.321300,9.999999,0.905342,A,0.442252,5.814018,3.517721,8.877414,1.191337
1,4,-93.247940,38.592617,54.511680,123.92817,48.573597,45.229984,7.246580,-17.264480,0.030932,-14.308875,94.821140,91.57980,-0.000614,0.001376,0.000588,-0.004100,0.000371,-0.003999,-0.000468,-0.000084,-0.000147,0.000706,-0.000207,0.001949,-25.950914,1.779193,-2.487668,-0.286266,5.677019,-0.396850,-0.618112,12.524258,1.614324,-1.088056,-5.308384,1.685836,0.007697,2.013834,-28.929590,57.018020,-27.310768,18.562830,-157.43951,14.462655,9.338835,-19.964195,-7.352732,-38.573727,1.0,171.01410,0.914076,-2.541605,-1.772064,-48.366608,-0.820012,1.806945,0.119447,12.426791,1.585607,-1.087751,-5.426009,1.469550,1.312910,14.475939,6.623713,2.462898,5.125846,2.512034,1.505335,12.085176,B,1.825361,4.247858,15.495952,A,3.108809,U,G,10.308044,10.476191,2.222223,10.595357,8.136964,4.003565e+00,18.140322,3.792658e+00,2.136160e+00,1.461561e+00,1.461561e+00,1.144708,2.436195,6.749117,33.998367,7.108153,7.503433,2.739240,2.739240,0.598896,AF,1.614802,2.963620,0,1.560137,1.589403,11.636386,1.992031,4.932127,3.554267,0.773906,0.198655,7.023803,18.036585,6.763110,GUV,1.845672e-07,C,1.826276,2.673322,5.043831,8.296139,C,C,A,8.068506,14.579479,0.642856,0.378418,4.205991,14.305766,12.934363,10.782008,9.156046,E,32.207340,154.04800,12.411608,12.411608,9.191265,1.379210,7.134018,G,DY,4.067039,5.330551,10.535108,2.412790,15.075894,2.923122e+00,5.478479,24.823215,5.189874,4.0,2,2.0,A,18.274548,9.516129,2.452166,12.053353,F,2,B,D,13.3333

In [36]:
scaler = StandardScaler()
df = scaler.fit_transform(df.drop(['ID','target','v107','v110','v112','v113','v125','v22','v24','v3',
                                   'v30','v31','v47','v52','v56','v66','v71','v72','v74','v75','v79','v91'], axis=1))

# Algoritmo LigthGBM

In [31]:
# Configurações Gerais

GENERATE_SUBMISSION_FILES = True
SUBMISSION_SUFIX = "_lgbm_v.1.0.6"
STRATIFIED_KFOLD = False
RANDOM_SEED = 737851
NUM_THREADS = 4
NUM_FOLDS = 10
EARLY_STOPPING = 1000

LIGHTGBM_PARAMS = {
    'boosting_type': 'goss',
    'n_estimators': 10000,
    'learning_rate': 0.005134,
    'num_leaves': 54,
    'max_depth': 10,
    'subsample_for_bin': 240000,
    'reg_alpha': 0.436193,
    'reg_lambda': 0.479169,
    'colsample_bytree': 0.508716,
    'min_split_gain': 0.024766,
    'subsample': 1,
    'is_unbalance': False,
    'silent':-1,
    'verbose':-1
}

In [32]:
# ------------------------- LIGHTGBM MODEL -------------------------

def run_model(data, categorical_feature = None):
    df = data[data['target'].notnull()]
    test = data[data['target'].isnull()]
    del_features = ['target']
    predictors = list(filter(lambda v: v not in del_features, df.columns))
    
    print("Train/valid shape: {}, test shape: {}".format(df.shape, test.shape))

    if not STRATIFIED_KFOLD:
        folds = KFold(n_splits= NUM_FOLDS, shuffle=True, random_state= RANDOM_SEED)
    else:
        folds = StratifiedKFold(n_splits= NUM_FOLDS, shuffle=True, random_state= RANDOM_SEED)

    # Hold oof predictions, test predictions, feature importance and training/valid auc
    oof_preds = np.zeros(df.shape[0])
    sub_preds = np.zeros(test.shape[0])
    importance_df = pd.DataFrame()
    eval_results = dict()

    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(df[predictors], df['target'])):
        train_x, train_y = df[predictors].iloc[train_idx], df['target'].iloc[train_idx]
        valid_x, valid_y = df[predictors].iloc[valid_idx], df['target'].iloc[valid_idx]

        params = {'random_state': RANDOM_SEED, 'nthread': NUM_THREADS}
        
        clf = LGBMClassifier(**{**params, **LIGHTGBM_PARAMS})
        
        if not categorical_feature.empty:
            clf.fit(train_x, train_y, 
                    eval_set=[(train_x, train_y), (valid_x, valid_y)],
                    eval_metric='logloss', 
                    verbose=400, 
                    early_stopping_rounds= EARLY_STOPPING)
        else:
            clf.fit(train_x, train_y, 
                    eval_set=[(train_x, train_y), (valid_x, valid_y)],
                    eval_metric='logloss', 
                    verbose=400, 
                    early_stopping_rounds=EARLY_STOPPING,
                    feature_name= list(df[predictors].columns), 
                    categorical_feature= categorical_feature)

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test[predictors], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        # Feature importance by GAIN and SPLIT
        fold_importance = pd.DataFrame()
        fold_importance["feature"] = predictors
        fold_importance["gain"] = clf.booster_.feature_importance(importance_type='gain')
        fold_importance["split"] = clf.booster_.feature_importance(importance_type='split')
        importance_df = pd.concat([importance_df, fold_importance], axis=0)
        eval_results['train_{}'.format(n_fold+1)]  = clf.evals_result_['training']['binary_logloss']
        eval_results['valid_{}'.format(n_fold+1)] = clf.evals_result_['valid_1']['binary_logloss']

        print('Fold %2d Log Loss : %.6f' % (n_fold + 1, log_loss(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full Log Loss score %.6f' % log_loss(df['target'], oof_preds))
    test['target'] = sub_preds.copy()

    # Get the average feature importance between folds
    mean_importance = importance_df.groupby('feature').mean().reset_index()
    mean_importance.sort_values(by= 'gain', ascending=False, inplace=True)
    
    
    # Save feature importance, test predictions and oof predictions as csv
    '''if GENERATE_SUBMISSION_FILES:

        # Save submission (test data) and feature importance
        submission = pd.read_csv('../dataset/sample_submission.csv')
        submission['PredictedProb'] = sub_preds.copy()
        submission.to_csv('../submission/submission{}.csv'.format(SUBMISSION_SUFIX), index=False)
        
        mean_importance.to_csv('feature_importance{}.csv'.format(SUBMISSION_SUFIX), index=False)
        plt.hist(submission.PredictedProb)
        plt.show()'''

        
    # Save feature importance, test predictions and oof predictions as csv
    if GENERATE_SUBMISSION_FILES:

        # Generate oof csv
        oof = pd.DataFrame()
        oof['ID'] = df['ID'].copy()
        df['predictions'] = oof_preds.copy()
        df['target'] = df['target'].copy()
        df.to_csv('oof{}.csv'.format(SUBMISSION_SUFIX), index=False)
        
        # Save submission (test data) and feature importance
        test[['ID', 'target']].to_csv('../submission/submission{}.csv'.format(SUBMISSION_SUFIX), index=False)
        mean_importance.to_csv('feature_importance{}.csv'.format(SUBMISSION_SUFIX), index=False)
        
        plt.hist(test['target'])
        plt.show()        
        
    return mean_importance

In [33]:
mean_importance = run_model(df, categorical_feats)

Train/valid shape: (114321, 183), test shape: (114393, 183)
Training until validation scores don't improve for 1000 rounds
[400]	training's binary_logloss: 0.463483	valid_1's binary_logloss: 0.478509
[800]	training's binary_logloss: 0.443518	valid_1's binary_logloss: 0.471611
[1200]	training's binary_logloss: 0.430875	valid_1's binary_logloss: 0.469619
[1600]	training's binary_logloss: 0.419956	valid_1's binary_logloss: 0.468488
[2000]	training's binary_logloss: 0.410351	valid_1's binary_logloss: 0.467832
[2400]	training's binary_logloss: 0.401111	valid_1's binary_logloss: 0.467203


KeyboardInterrupt: 

# Stacking

In [14]:
class StackingEstimator(BaseEstimator, TransformerMixin):
    
    def __init__(self, estimator):
        self.estimator = estimator

    def fit(self, X, y=None, **fit_params):
        self.estimator.fit(X, y, **fit_params)
        return self
    
    def transform(self, X):
        X = check_array(X)
        X_transformed = np.copy(X)
        
        # add class probabilities as a synthetic feature
        if issubclass(self.estimator.__class__, ClassifierMixin) and hasattr(self.estimator, 'predict_proba'):
            X_transformed = np.hstack((self.estimator.predict_proba(X), X))

        # add class prodiction as a synthetic feature
        X_transformed = np.hstack((np.reshape(self.estimator.predict(X), (-1, 1)), X_transformed))

        return X_transformed

In [ ]:
    
stacked_pipeline = make_pipeline(
    StackingEstimator(estimator=LassoLarsCV(normalize=True)),
    StackingEstimator(estimator=GradientBoostingRegressor(learning_rate=0.001, 
                                                          loss="huber", 
                                                          max_depth=4, 
                                                          max_features=0.55, 
                                                          min_samples_leaf=17, 
                                                          min_samples_split=14, 
                                                          subsample=0.7)),
    LassoLarsCV()
)

# Separando features preditoras e target
X = df.drop(['ID','target'], axis=1)
y = df['target']

stacked_pipeline.fit(X, y)
results = stacked_pipeline.predict(X)

#LogLoss Score on the entire Train data when averaging
print('Log Loss score on train data:')
print(log_loss(y, stacked_pipeline.predict(X)*0.1))
print(log_loss(y, stacked_pipeline.predict(X)*0.2))
print(log_loss(y, stacked_pipeline.predict(X)*0.3))
print(log_loss(y, stacked_pipeline.predict(X)*0.4))